### Case study of the use of Random Forest via Spark for the prediction of the cause of the end of a certain chess game.

#### First, we will import the necessary packages and the csv containing the detaset courtesy of Mitchell J (https://mitchelljolly.com/)

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, 
                                StringIndexer, OneHotEncoder)
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
spark = SparkSession.builder.appName('chessApp').getOrCreate()
data = spark.read.csv('/FileStore/tables/games.csv',
                      inferSchema=True,
                      header=True)

#### Now, we can get information about the dataset

In [0]:
data.show()

+--------+-----+----------+------------+-----+--------------+------+--------------+------------------+------------+------------------+------------+--------------------+-----------+--------------------+-----------+
 id|rated|created_at|last_move_at|turns|victory_status|winner|increment_code| white_id|white_rating| black_id|black_rating| moves|opening_eco| opening_name|opening_ply|
+--------+-----+----------+------------+-----+--------------+------+--------------+------------------+------------+------------------+------------+--------------------+-----------+--------------------+-----------+
TZJHLljE|false|1.50421E12| 1.50421E12| 13| outoftime| white| 15+2| bourgris| 1500| a-00| 1191|d4 d5 c4 c6 cxd5 ...| D10|Slav Defense: Exc...| 5|
l1NXvwaE| true|1.50413E12| 1.50413E12| 16| resign| black| 5+10| a-00| 1322| skinnerua| 1261|d4 Nc6 e4 e5 f4 f...| B00|Nimzowitsch Defen...| 4|
mIICvQHh| true|1.50413E12| 1.50413E12| 61| mate| white| 5+10| ischia| 1496| a-00| 1500|e4 e5 d3 d6 Be3 c...| C20|King's Pawn Game:...| 3|
kWKvrqYL| true|1.50411E12| 1.50411E12| 61| mate| white| 20+0| daniamurashov| 1439| adivanov2009| 1454|d4 d5 Nf3 Bf5 Nc3...| D02|Queen's Pawn Game...| 3|
9tXo1AUZ| true|1.50403E12| 1.50403E12| 95| mate| white| 30+3| nik221107| 1523| adivanov2009| 1469|e4 e5 Nf3 d6 d4 N...| C41| Philidor Defense| 5|
MsoDV9wj|false|1.50424E12| 1.50424E12| 5| draw| draw| 10+0| trelynn17| 1250| franklin14532| 1002| e4 c5 Nf3 Qa5 a3| B27|Sicilian Defense:...| 4|
qwU9rasv| true|1.50423E12| 1.50423E12| 33| resign| white| 10+0| capa_jr| 1520|daniel_likes_chess| 1423|d4 d5 e4 dxe4 Nc3...| D00|Blackmar-Diemer G...| 10|
RVN0N3VK|false|1.50368E12| 1.50368E12| 9| resign| black| 15+30|daniel_likes_chess| 1413| soultego| 2108|e4 Nc6 d4 e5 d5 N...| B00|Nimzowitsch Defen...| 5|
dwF3DJHO| true|1.50351E12| 1.50351E12| 66| resign| black| 15+0| ehabfanri| 1439|daniel_likes_chess| 1392|e4 e5 Bc4 Nc6 Nf3...| C50|Italian Game: Sch...| 6|
afoMwnLg| true|1.50344E12| 1.50344E12| 119| mate| white| 10+0|daniel_likes_chess| 1381| mirco25| 1209|e4 d5 exd5 Qxd5 N...| B01|Scandinavian Defe...| 4|
HgKLWPsz|false|1.50335E12| 1.50335E12| 39| mate| white| 20+60|daniel_likes_chess| 1381| anaissac| 1272|e3 e6 d4 d6 Bd3 c...| A00|Van't Kruijs Opening| 1|
Vf5fKWzI|false|1.50335E12| 1.50335E12| 38| resign| black| 20+60|daniel_likes_chess| 1381| subham777| 1867|e4 e6 d4 d5 e5 c5...| C02|French Defense: A...| 9|
HRti5mKv|false|1.50335E12| 1.50335E12| 60| resign| black| 5+40|daniel_likes_chess| 1381| roman12342005| 1936|e4 e6 Nf3 d5 exd5...| C00|French Defense: K...| 3|
2fEjSei6|false|1.50334E12| 1.50334E12| 31| resign| black| 8+0|daniel_likes_chess| 1381| alkhan| 1607|e4 e6 Qh5 g6 Qe5 ...| C00| French Defense #2| 2|
u7i6dOaJ|false|1.50409E12| 1.5041E12| 31| mate| white| 15+15| shivangithegenius| 1094| sureka_akshat| 1141|e4 e5 Nf3 Nc6 Bc4...| C50|Four Knights Game...| 8|
guanvMR5|false|1.50409E12| 1.50409E12| 43| resign| black| 15+15| sureka_akshat| 1141| shivangithegenius| 1094|e4 e5 Nf3 Nc6 Bc4...| C57|Italian Game: Two...| 7|
PmpkWkmm|false|1.50409E12| 1.50409E12| 52| resign| black| 15+15| shivangithegenius| 1094| sureka_akshat| 1141|e4 e5 Nf3 Nc6 Bc4...| C50|Four Knights Game...| 8|
EwaK0IsE|false|1.50401E12| 1.50401E12| 66| mate| black| 15+16| sureka_akshat| 1141| shivangithegenius| 1094|e4 e5 Nf3 Nc6 Bc4...| C50|Four Knights Game...| 8|
yrSDozT3|false|1.50401E12| 1.50401E12| 101| resign| black| 15+15| shivangithegenius| 1094| slam_ment| 1300|e4 e5 Nf3 d6 Bc4 ...| C41| Philidor Defense #3| 5|
x31mXlvc|false|1.50376E12| 1.50376E12| 25| resign| white| 11+0| g-ios| 1500| shivangithegenius| 1094|d4 d5 h3 Nc6 Nf3 ...| D00| Queen's Pawn Game| 2|
+--------+-----+----------+------------+-----+--------------+------+--------------+------------------+------------+------------------+------------+--------------------+-----------+--------------------+-----------+
only showing top 20 rows

In [0]:
data.printSchema()

root
-- id: string (nullable = true)
-- rated: boolean (nullable = true)
-- created_at: double (nullable = true)
-- last_move_at: double (nullable = true)
-- turns: integer (nullable = true)
-- victory_status: string (nullable = true)
-- winner: string (nullable = true)
-- increment_code: string (nullable = true)
-- white_id: string (nullable = true)
-- white_rating: integer (nullable = true)
-- black_id: string (nullable = true)
-- black_rating: integer (nullable = true)
-- moves: string (nullable = true)
-- opening_eco: string (nullable = true)
-- opening_name: string (nullable = true)
-- opening_ply: integer (nullable = true)

#### We are going to stay only with the columns that interest us

In [0]:
data_selected = data.select('rated', 'turns', 'victory_status', 'winner', 'increment_code', 'white_rating', 'black_rating', 'opening_name')

In [0]:
data_selected.show()

+-----+-----+--------------+------+--------------+------------+------------+--------------------+
rated|turns|victory_status|winner|increment_code|white_rating|black_rating| opening_name|
+-----+-----+--------------+------+--------------+------------+------------+--------------------+
false| 13| outoftime| white| 15+2| 1500| 1191|Slav Defense: Exc...|
 true| 16| resign| black| 5+10| 1322| 1261|Nimzowitsch Defen...|
 true| 61| mate| white| 5+10| 1496| 1500|King's Pawn Game:...|
 true| 61| mate| white| 20+0| 1439| 1454|Queen's Pawn Game...|
 true| 95| mate| white| 30+3| 1523| 1469| Philidor Defense|
false| 5| draw| draw| 10+0| 1250| 1002|Sicilian Defense:...|
 true| 33| resign| white| 10+0| 1520| 1423|Blackmar-Diemer G...|
false| 9| resign| black| 15+30| 1413| 2108|Nimzowitsch Defen...|
 true| 66| resign| black| 15+0| 1439| 1392|Italian Game: Sch...|
 true| 119| mate| white| 10+0| 1381| 1209|Scandinavian Defe...|
false| 39| mate| white| 20+60| 1381| 1272|Van't Kruijs Opening|
false| 38| resign| black| 20+60| 1381| 1867|French Defense: A...|
false| 60| resign| black| 5+40| 1381| 1936|French Defense: K...|
false| 31| resign| black| 8+0| 1381| 1607| French Defense #2|
false| 31| mate| white| 15+15| 1094| 1141|Four Knights Game...|
false| 43| resign| black| 15+15| 1141| 1094|Italian Game: Two...|
false| 52| resign| black| 15+15| 1094| 1141|Four Knights Game...|
false| 66| mate| black| 15+16| 1141| 1094|Four Knights Game...|
false| 101| resign| black| 15+15| 1094| 1300| Philidor Defense #3|
false| 25| resign| white| 11+0| 1500| 1094| Queen's Pawn Game|
+-----+-----+--------------+------+--------------+------------+------------+--------------------+
only showing top 20 rows

#### Next, we treat those variables that are categorical.

In [0]:
victory_Indexer = StringIndexer(inputCol='victory_status', outputCol='victory_status_Index')
data_selected = victory_Indexer.fit(data_selected).transform(data_selected)

In [0]:
winner_Indexer = StringIndexer(inputCol='winner', outputCol='winner_Index')
winner_Vec = OneHotEncoder(inputCol='winner_Index', outputCol='winner_Vec')
data_selected = winner_Indexer.fit(data_selected).transform(data_selected)
data_selected = winner_Vec.fit(data_selected).transform(data_selected)

In [0]:
opening_Indexer = StringIndexer(inputCol='opening_name', outputCol='opening_name_Index')
opening_Vec = OneHotEncoder(inputCol='opening_name_Index', outputCol='opening_name_Vec')
data_selected = opening_Indexer.fit(data_selected).transform(data_selected)
data_selected = opening_Vec.fit(data_selected).transform(data_selected)

In [0]:
increment_Indexer = StringIndexer(inputCol='increment_code', outputCol='increment_code_Index')
increment_Vec = OneHotEncoder(inputCol='increment_code_Index', outputCol='increment_code_Vec')
data_selected = increment_Indexer.fit(data_selected).transform(data_selected)
data_selected = increment_Vec.fit(data_selected).transform(data_selected)

#### Applying the VectorAssembler

In [0]:
data_selected.columns

Out[12]: ['rated',
 'turns',
 'victory_status',
 'winner',
 'increment_code',
 'white_rating',
 'black_rating',
 'opening_name']

In [0]:
assembler = VectorAssembler(inputCols=['rated','turns','winner_Vec','increment_code_Vec','white_rating', 'black_rating', 'opening_name_Vec'],
                           outputCol='features')

In [0]:
final_data = assembler.transform(data_selected)

In [0]:
final_data.select('victory_status_Index','features').show()

+--------------------+--------------------+
victory_status_Index| features|
+--------------------+--------------------+
 2.0|(1881,[1,2,25,403...|
 0.0|(1881,[0,1,3,19,4...|
 1.0|(1881,[0,1,2,19,4...|
 1.0|(1881,[0,1,2,12,4...|
 1.0|(1881,[0,1,2,155,...|
 3.0|(1881,[1,4,403,40...|
 0.0|(1881,[0,1,2,4,40...|
 0.0|(1881,[1,3,92,403...|
 0.0|(1881,[0,1,3,5,40...|
 1.0|(1881,[0,1,2,4,40...|
 1.0|(1881,[1,2,73,403...|
 0.0|(1881,[1,3,73,403...|
 0.0|(1881,[1,3,217,40...|
 0.0|(1881,[1,3,9,403,...|
 1.0|(1881,[1,2,6,403,...|
 0.0|(1881,[1,3,6,403,...|
 0.0|(1881,[1,3,6,403,...|
 1.0|(1881,[1,3,154,40...|
 0.0|(1881,[1,3,6,403,...|
 0.0|(1881,[1,2,52,403...|
+--------------------+--------------------+
only showing top 20 rows

#### Specifying the parameters of the Random Forest

In [0]:
random_forest_classifier = RandomForestClassifier(labelCol='victory_status_Index', featuresCol='features')

#### Splitting into train and test data

In [0]:
train_data ,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
train_data.show()

+-----+-----+--------------+------+--------------+------------+------------+--------------------+--------------------+
rated|turns|victory_status|winner|increment_code|white_rating|black_rating| opening_name|victory_status_Index|
+-----+-----+--------------+------+--------------+------------+------------+--------------------+--------------------+
false| 1| outoftime| white| 5+15| 2100| 1404| King's Pawn| 2.0|
false| 1| outoftime| white| 5+5| 1500| 2723| King's Pawn| 2.0|
false| 2| outoftime| black| 10+0| 1761| 1579| Sicilian Defense| 2.0|
false| 2| outoftime| black| 15+0| 1038| 1331|Scandinavian Defense| 2.0|
false| 2| outoftime| black| 15+0| 1500| 1382| Queen's Pawn Game| 2.0|
false| 2| outoftime| black| 15+0| 1500| 1586| King's Pawn Game| 2.0|
false| 2| outoftime| black| 18+18| 1500| 1500| Sicilian Defense| 2.0|
false| 2| resign| black| 15+15| 2156| 2476| Queen's Pawn Game| 0.0|
false| 2| resign| black| 15+5| 1401| 1612| Pirc Defense #4| 0.0|
false| 2| resign| black| 25+17| 1149| 1672|Scandinavian Defense| 0.0|
false| 2| resign| black| 30+0| 1091| 1500|Van't Kruijs Opening| 0.0|
false| 2| resign| black| 5+30| 1264| 1507| Pirc Defense #4| 0.0|
false| 2| resign| black| 8+0| 2021| 1949| King's Pawn Game| 0.0|
false| 2| resign| white| 15+0| 1866| 1500| King's Pawn Game| 0.0|
false| 2| resign| white| 20+10| 1049| 1275|Van't Kruijs Opening| 0.0|
false| 2| resign| white| 30+20| 2430| 1566| Caro-Kann Defense| 0.0|
false| 2| resign| white| 5+5| 2270| 1962| Englund Gambit| 0.0|
false| 2| resign| white| 8+0| 1500| 1784| French Defense #2| 0.0|
false| 3| draw| draw| 60+0| 1990| 1779|Zukertort Opening...| 3.0|
false| 3| outoftime| white| 10+0| 1102| 1069|Scandinavian Defense| 2.0|
+-----+-----+--------------+------+--------------+------------+------------+--------------------+--------------------+
only showing top 20 rows

#### Training and testing the model

In [0]:
fit_model = random_forest_classifier.fit(train_data)

In [0]:
results = fit_model.transform(test_data)

#### Finally, we can get the acuraccy or our model and the importance of each variable

In [0]:
my_eval = MulticlassClassificationEvaluator(labelCol='victory_status_Index', metricName='accuracy')

In [0]:
results_evaluated = my_eval.evaluate(results)
print(results_evaluated)

0.5531123139377537